In [1]:
from reader import readShortVideo
from reader import getVideoList
import matplotlib.pyplot as plt
from os import listdir
import os
import pandas as pd
import numpy as np
import pickle

import torchvision
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
%matplotlib inline
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


# read pre-defined feature and train RNN

In [2]:
with open("../train_features_d12.pkl", "rb") as f:
    train_features = pickle.load(f)
with open("../valid_features_d12.pkl", "rb") as f:
    valid_features = pickle.load(f)
    
with open("../train_y.pkl", "rb") as f:
    train_y = pickle.load(f)
with open("../valid_y.pkl", "rb") as f:
    valid_y = pickle.load(f)

In [27]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size=512, n_layers=2, dropout=0.5):
        super(GRU, self).__init__()
        self.hidden_size =  hidden_size
        self.lstm = nn.LSTM(input_size, self.hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=False)
        self.bn_0 = nn.BatchNorm1d(self.hidden_size)
        self.fc_1 = nn.Linear(self.hidden_size, int(self.hidden_size/2))
        self.bn_1 = nn.BatchNorm1d(int(self.hidden_size/2))
        self.fc_2 = nn.Linear(int(self.hidden_size), 11)
        self.softmax = nn.Softmax(1)
        self.relu = nn.ReLU()
    def forward(self, padded_sequence, input_lengths, hidden=None):
        packed = torch.nn.utils.rnn.pack_padded_sequence(padded_sequence, input_lengths)
        outputs, (hn,cn) = self.lstm(packed, hidden) # output: (seq_len, batch, hidden*n_dir)
#         outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
#         outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs (1, batch, hidden)
        outputs = self.bn_0(hn[-1])
        outputs = self.softmax(self.fc_2(outputs))
        return outputs


In [28]:
def single_batch_padding(train_X_batch, train_y_batch, test = False):
    if test==True:
        padded_sequence = nn.utils.rnn.pad_sequence(train_X_batch)
        label = torch.LongTensor(train_y_batch)
        length = [len(train_X_batch[0])]
    else:
        length = [len(x) for x in train_X_batch]
        perm_index = np.argsort(length)[::-1]

        # sort by sequence length
        train_X_batch = [train_X_batch[i] for i in perm_index]
        length = [len(x) for x in train_X_batch]
        padded_sequence = nn.utils.rnn.pad_sequence(train_X_batch)
        label = torch.LongTensor(np.array(train_y_batch)[perm_index])
    return padded_sequence, label, length

In [29]:
input_valid_X, input_valid_y, valid_lengths = single_batch_padding([valid_features[0]], [valid_y[0]],
                                                                  test=True)

In [30]:
feature_size = 1024*7*7
model = rnn = GRU(feature_size,hidden_size=512).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
BATCH_SIZE = 64
loss_function = nn.CrossEntropyLoss()
max_accuracy = 0
model.train()
for epoch in range(20):
    print("Epoch:", epoch+1)
    CE_loss = 0.0
    total_length = len(train_features)
    # shuffle
    perm_index = np.random.permutation(len(train_features))
    train_X_sfl = [ train_features[i] for i in perm_index]
    train_y_sfl = np.array(train_y)[perm_index]
    # construct training batch
    for index in range(0,total_length ,BATCH_SIZE):
        if index+BATCH_SIZE > total_length:
            break
            
        # zero the parameter gradients
        optimizer.zero_grad()
        input_X = train_X_sfl[index:index+BATCH_SIZE]
        input_y = train_y_sfl[index:index+BATCH_SIZE]
        
        # pad the sequence
        input_X, input_y, length = single_batch_padding(input_X, input_y)
        
        # use GPU
        input_X = input_X.cuda()
        # forward + backward + optimize
        output = model(input_X, length)
        loss = loss_function(output, input_y.cuda())
        loss.backward()
        optimizer.step()
        CE_loss += loss.cpu().data.numpy()
    print("training loss",CE_loss)
    
    # validation
    same_difference = []
    with torch.no_grad():
        model.eval()
        for i in range(len(valid_y)):
            input_valid_X, input_valid_y, valid_lengths = single_batch_padding([valid_features[i]], 
                                                                               [valid_y[i]],
                                                                               test=True)
            output = model(input_valid_X.cuda(),valid_lengths)
            output_label = torch.argmax(output,1).cpu().data
            same_difference.append((output_label == input_valid_y).numpy())
        accuracy = np.mean(same_difference)
        print("validation accuracy: ",accuracy)
    if accuracy > max_accuracy:
        max_accuracy = accuracy
        torch.save(model.state_dict(), "../models/RNN_FC_model.pkt")
    model.train()

Epoch: 1
training loss 116.47475147247314
validation accuracy:  0.28433268858800775
Epoch: 2
training loss 108.52313160896301
validation accuracy:  0.3849129593810445
Epoch: 3
training loss 103.10326600074768
validation accuracy:  0.44874274661508706
Epoch: 4
training loss 99.2244381904602
validation accuracy:  0.4835589941972921
Epoch: 5
training loss 95.94136488437653
validation accuracy:  0.4738878143133462
Epoch: 6
training loss 93.1365247964859
validation accuracy:  0.5067698259187621
Epoch: 7
training loss 91.02155685424805
validation accuracy:  0.49709864603481624
Epoch: 8
training loss 89.38884377479553
validation accuracy:  0.4990328820116054
Epoch: 9
training loss 87.75367951393127
validation accuracy:  0.5183752417794971
Epoch: 10
training loss 86.91279792785645
validation accuracy:  0.5183752417794971
Epoch: 11
training loss 86.06727576255798
validation accuracy:  0.5067698259187621
Epoch: 12
training loss 85.13175427913666
validation accuracy:  0.5087040618955513
Epoch: 13

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# 先加 hidden dimension
# 加 fc layer
# 調 dropout

In [ ]:
?nn.LSTM()